In [1]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:

# Load and prepare data
df = pd.read_csv("HR_Analytics.csv")
df['YearsWithCurrManager'].fillna(df['YearsWithCurrManager'].median(), inplace=True)


C:\Users\badar\AppData\Local\Temp\ipykernel_26648\1040038394.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['YearsWithCurrManager'].fillna(df['YearsWithCurrManager'].median(), inplace=True)


In [5]:
# Encode categorical features
categorical_cols = df.select_dtypes(include='object').columns.tolist()
categorical_cols.remove('EmpID')
categorical_cols.remove('Attrition')

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [7]:
# Encode target
target_encoder = LabelEncoder()
df['Attrition'] = target_encoder.fit_transform(df['Attrition'])

# Save JobRole encoder
jobrole_encoder = label_encoders['JobRole']
df['JobRole_Name'] = jobrole_encoder.inverse_transform(df['JobRole'])


In [9]:
# Drop ID column
df.drop(columns=['EmpID'], inplace=True)


In [11]:
# Train model
X = df.drop(columns=['Attrition', 'JobRole_Name'])
y = df['Attrition']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [15]:
#RandomForest

model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

In [17]:
# Prepare top 5 job roles with highest demand
job_demand = df.groupby('JobRole_Name')['Attrition'].agg(['count', 'sum']).reset_index()
job_demand['AttritionRate'] = job_demand['sum'] / job_demand['count']
job_demand['DemandScore'] = 1 - job_demand['AttritionRate']
job_demand = job_demand.sort_values(by='DemandScore', ascending=False).head(5)


In [21]:
# Prediction function
def predict_attrition(age, job_level, monthly_income, years_at_company, job_role):
    job_role_encoded = label_encoders['JobRole'].transform([job_role])[0]
    input_df = pd.DataFrame([[age, job_level, monthly_income, years_at_company, job_role_encoded]],
                            columns=['Age', 'JobLevel', 'MonthlyIncome', 'YearsAtCompany', 'JobRole'])
        # Fill in zeros for unused columns
    for col in X.columns:
        if col not in input_df.columns:
            input_df[col] = 0
    input_df = input_df[X.columns]  # Reorder

    input_scaled = scaler.transform(input_df)
    prob = model.predict_proba(input_scaled)[0][1]
    prediction = "✅ Likely to Stay" if prob < 0.5 else "❌ Likely to Leave"
    return prediction, round(prob * 100, 2), job_demand


In [23]:
# Interface
inputs = [
    gr.Number(label="Age", value=30),
    gr.Slider(1, 5, step=1, label="Job Level"),
    gr.Number(label="Monthly Income", value=5000),
    gr.Slider(0, 40, step=1, label="Years at Company"),
    gr.Dropdown(choices=list(jobrole_encoder.classes_), label="Job Role")
]

outputs = [
    gr.Text(label="Depletion Prediction"),
    gr.Number(label="Confidence (% Leaving)"),
    gr.Dataframe(label="Top 5 In-Demand Job Roles (Lowest Attrition)")
]

gr.Interface(
    fn=predict_attrition,
    inputs=inputs,
    outputs=outputs,
    title="🧠 Staff Depletion",
    description="Predict if an employee is likely to leave and view top job roles with highest retention rates."
).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
